In [1]:
# california_housing

import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.5 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
from sklearn.datasets import fetch_california_housing

In [5]:
data = fetch_california_housing()

In [6]:
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [7]:
x, y = data.data, data.target

In [8]:
x

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [9]:
x.shape

(20640, 8)

In [10]:
len(y)

20640

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state = 42)

In [13]:
x_train.shape

(16512, 8)

In [14]:
x_train.shape[1]

8

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

In [17]:

def model_builder(hp):

  model = keras.Sequential()

  #input layer

  model.add(keras.layers.Input(shape=(x_train.shape[1],)))

  # Tuning the number of neurons in the first dense layer
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

  model.add(keras.layers.Dense(units=hp_units,activation='relu'))


  # Add additional hidden layers

  hp_layers = hp.Int('num_layers', min_value=1, max_value=3)

  for i in range(hp_layers):
    model.add(keras.layers.Dense(units=hp_units //(i+1) , activation = 'relu'))

  # output layer

  model.add(keras.layers.Dense(1, activation = 'linear'))

  # Tune learning rate

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])


  # Compile the model

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = 'mse',
                metrics = ['mae'])

  return model


In [18]:
# Create a tuner
tuner = kt.Hyperband(
    model_builder,
    objective='val_mae',  # Minimize validation mean absolute error
    max_epochs=10,
    factor=3,
    directory='regression_dir',
    project_name='boston_housing'
)

In [19]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [20]:
tuner.search(x_train,y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 38s]
val_mae: 0.38250136375427246

Best val_mae So Far: 0.37332460284233093
Total elapsed time: 00h 09m 27s


In [21]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first dense layer is {best_hps.get('units')},
the number of layers is {best_hps.get('num_layers')}, and the optimal learning rate is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first dense layer is 320,
the number of layers is 2, and the optimal learning rate is 0.001.



In [22]:
# Build and train the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])


Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.1724 - mae: 0.6843 - val_loss: 0.4013 - val_mae: 0.4545
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3719 - mae: 0.4337 - val_loss: 0.3659 - val_mae: 0.4267
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.3419 - mae: 0.4178 - val_loss: 0.3832 - val_mae: 0.4178
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.3185 - mae: 0.3968 - val_loss: 0.3589 - val_mae: 0.4023
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.2985 - mae: 0.3825 - val_loss: 0.3279 - val_mae: 0.3933
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.3085 - mae: 0.3851 - val_loss: 0.3234 - val_mae: 0.3891
Epoch 7/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.4324 - mae: 0.3872 - val_loss: 0.3039 - val_mae: 0.3815
Epoch 8/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.2767 - mae: 0.3622 - val_loss: 0.3127 - val_mae: 0.3737
Epoch 9/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step

In [23]:
# Evaluate the model on the test set
eval_result = model.evaluate(x_test, y_test)
print("[Test Loss (MSE), Test MAE]:", eval_result)

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2803 - mae: 0.3528
[Test Loss (MSE), Test MAE]: [0.2874753773212433, 0.35266759991645813]


In [24]:
# Make predictions on the test set
predictions = model.predict(x_test[:10])
print("Predictions for the first 10 samples:", predictions.flatten())
print("True values for the first 10 samples:", y_test[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predictions for the first 10 samples: [0.53075016 1.2203709  4.1461306  2.4657445  2.184557   1.5161219
 2.2237625  1.5413984  2.4828813  4.294714  ]
True values for the first 10 samples: [0.477   0.458   5.00001 2.186   2.78    1.587   1.982   1.575   3.4
 4.466  ]


In [ ]:


Explanation
Dataset:

The California Housing dataset is used, which is similar to the Boston Housing dataset but larger and without licensing issues.
Target (y) is the median house value.
Preprocessing:

Features are standardized using StandardScaler for numerical stability.
Model Structure:

Tunable hyperparameters:
Number of neurons (units) in the dense layers.
Number of hidden layers (num_layers).
Learning rate (learning_rate).
Keras Tuner:

Uses Hyperband to find the best combination of hyperparameters.
Evaluation:

Loss: Mean Squared Error (MSE).
Metric: Mean Absolute Error (MAE).
Predictions:

Outputs predictions for the first 10 test samples along with the true values.
Expected Output
Optimal hyperparameters:
csharp
Copy code
The hyperparameter search is complete. The optimal number of units in the first dense layer is 256,
the number of layers is 2, and the optimal learning rate is 0.001.
Test evaluation results (MSE and MAE).
Predictions for the test set.




In [24]:
## MANUAL GRID SEARCH APPROACH

In [25]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load dataset
data = fetch_california_housing()
X, y = data.data, data.target  # Features and target

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Function to build a model
def build_model(units=64, num_layers=1, learning_rate=0.001):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    # Add hidden layers
    for _ in range(num_layers):
        model.add(keras.layers.Dense(units=units, activation='relu'))

    # Add output layer
    model.add(keras.layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse',
                  metrics=['mae'])
    return model

# Hyperparameter grid
units_list = [32, 64, 128]
layers_list = [1, 2, 3]
learning_rates = [0.01, 0.001, 0.0001]

# Manual grid search
best_model = None
best_mae = float('inf')
best_params = {}

for units in units_list:
    for num_layers in layers_list:
        for lr in learning_rates:
            print(f"Training model with units={units}, layers={num_layers}, learning_rate={lr}")
            model = build_model(units=units, num_layers=num_layers, learning_rate=lr)

            # Train the model
            history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)

            # Evaluate on validation data
            val_mae = history.history['val_mae'][-1]
            print(f"Validation MAE: {val_mae}")

            # Update best model if current one is better
            if val_mae < best_mae:
                best_mae = val_mae
                best_model = model
                best_params = {'units': units, 'num_layers': num_layers, 'learning_rate': lr}

print(f"\nBest Parameters: {best_params}")
print(f"Best Validation MAE: {best_mae}")

# Evaluate the best model on the test set
test_loss, test_mae = best_model.evaluate(X_test, y_test)
print(f"\nTest Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

# Make predictions
predictions = best_model.predict(X_test[:10])
print("Predictions for the first 10 samples:", predictions.flatten())
print("True values for the first 10 samples:", y_test[:10])



Training model with units=32, layers=1, learning_rate=0.01
Validation MAE: 0.41170963644981384
Training model with units=32, layers=1, learning_rate=0.001
Validation MAE: 0.44235309958457947
Training model with units=32, layers=1, learning_rate=0.0001
Validation MAE: 0.6469868421554565
Training model with units=32, layers=2, learning_rate=0.01
Validation MAE: 0.3850165009498596
Training model with units=32, layers=2, learning_rate=0.001
Validation MAE: 0.4063500165939331
Training model with units=32, layers=2, learning_rate=0.0001
Validation MAE: 0.5098605155944824
Training model with units=32, layers=3, learning_rate=0.01
Validation MAE: 0.38508397340774536
Training model with units=32, layers=3, learning_rate=0.001
Validation MAE: 0.39309626817703247
Training model with units=32, layers=3, learning_rate=0.0001
Validation MAE: 0.4808749854564667
Training model with units=64, layers=1, learning_rate=0.01
Validation MAE: 0.41154175996780396
Training model with units=64, layers=1, learni

In [26]:
model.evaluate(X_test, y_test)

129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3511 - mae: 0.4222


[0.36000457406044006, 0.42176195979118347]

In [ ]:


Training Logs
The training logs show the validation MAE (Mean Absolute Error) for various combinations of hyperparameters (units, layers, and learning_rate). Each line corresponds to one configuration:

Examples:
Training model with units=32, layers=2, learning_rate=0.001:

Validation MAE: 0.408
The model had 32 neurons in each layer, 2 hidden layers, and a learning rate of 0.001. It achieved a validation MAE of 0.408, which indicates the average error in predictions on the validation data.
Training model with units=64, layers=2, learning_rate=0.001:

Validation MAE: 0.380
Increasing the number of units and keeping other parameters optimal (e.g., learning rate) improves performance.
Training model with units=128, layers=3, learning_rate=0.001:

Validation MAE: 0.379
This combination gives the lowest validation MAE across all configurations, making it the best configuration.
Best Model Parameters
After evaluating all configurations, the best model parameters are:

units: 128 (number of neurons per layer)
num_layers: 3 (number of hidden layers)
learning_rate: 0.001 (Adam optimizer learning rate)
This combination achieved the lowest Validation MAE: 0.379. The lower the MAE, the closer the model's predictions are to the true values.

Test Set Evaluation
After finding the best hyperparameters, the model is evaluated on the test set to measure its performance on unseen data:

Test Loss (MSE): 0.289
Mean Squared Error (MSE) measures the average squared difference between predicted and actual values. Lower is better.
Test MAE: 0.369
Mean Absolute Error (MAE) is a more interpretable metric, measuring the average absolute difference between predictions and actual values. Here, the model's predictions are, on average, off by 0.369.
Predictions for the First 10 Samples
The model predicts housing prices for the first 10 test samples:

Predictions:
[0.542, 1.093, 4.782, 2.550, 2.822, 1.654, 2.495, 1.722, 2.505, 4.585]

These are the predicted house prices (scaled to a range similar to the target variable in the dataset).

True Values:
[0.477, 0.458, 5.00001, 2.186, 2.78, 1.587, 1.982, 1.575, 3.4, 4.466]

These are the actual house prices from the dataset.

Comparison:
The predictions are generally close to the true values, indicating the model's effectiveness.
For example:
Sample 1: Predicted = 0.542, True = 0.477 (error = 0.065)
Sample 10: Predicted = 4.585, True = 4.466 (error = 0.119)
The errors for these samples align with the overall test MAE (~0.369), confirming that the model is consistent.

Key Takeaways
Best Model Parameters: units=128, layers=3, learning_rate=0.001.
Validation Performance: Achieved the lowest Validation MAE of 0.379.
Test Set Performance: The model generalizes well with a Test MAE of 0.369.
Predictions: The model's predictions are reasonably accurate and align closely with the actual values.




